# Plot Y1 blinded chains
This jupyter notebook contains some code to make plots of the bestfits and contours for each of the Y1 blinded samples, looping over each tracer and cap.

In [1]:
# Import some necessary modules
import os
import sys
import pickle
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from chainconsumer import ChainConsumer     

sys.path.append("../../../Barry/")     # Change this so that it points to where you have Barry installed

from barry.samplers import DynestySampler
from barry.config import setup
from barry.models import PowerBeutler2017, CorrBeutler2017
from barry.datasets.dataset_power_spectrum import PowerSpectrum_DESI_KP4
from barry.datasets.dataset_correlation_function import CorrelationFunction_DESI_KP4
from barry.fitter import Fitter
from barry.utils import weighted_avg_and_cov

# Read in the fitter class to get all the info on the fit
pfn = "./plots/run_fits/output/run_fits.fitter.pkl"
with open(pfn, 'rb') as pickle_file:
    fitter = pickle.load(pickle_file)

This code segment reads in the chains, plots the bestfit model vs. data and prepares stuff for contour plots and summary files

In [2]:
%%capture
logging.info("Creating plots")

# The different tracers and redshift bins
tracers = {'BGS_BRIGHT-21.5': [[0.1, 0.4]], 
           'LRG': [[0.4, 0.6], [0.6, 0.8], [0.8, 1.1]], 
           'ELG_LOPnotqso': [[0.8, 1.1], [1.1, 1.6]],
           'QSO': [[0.8, 2.1]]}

# The different sky areas
caps = ["NGC", "SGC", "GCcomb"]

# Set up some convenient colours
cs = ["#74B666","#38AA88","#2899A0","#5783A4","#7E6990","#8F526C","#894446"]

# Let's group together the three caps so we can overplot them on the same contour plot
datanames = [f"{t}_{zs[0]}_{zs[1]}" for t in tracers for i, zs in enumerate(tracers[t])]
cfull = [ChainConsumer() for d in datanames]
csmall = [ChainConsumer() for d in datanames]

# Loop over all the chains
output = []
for posterior, weight, chain, evidence, model, data, extra in fitter.load():

    # Match the chain to the datanames list created above
    subname = extra["name"].split()[3]
    tracer = ("_").join(subname.split("_")[:-3])
    cap, zmin, zmax = subname.split("_")[-3:]
    chainname = f"{tracer}_{zmin}_{zmax}"
    dataname_index = [i for i, n in enumerate(datanames) if chainname in n][0]
    
    # Store the chain in a dictionary with parameter names
    df = pd.DataFrame(chain, columns=model.get_labels())

    # Compute alpha_par and alpha_perp for each point in the chain
    alpha_par, alpha_perp = model.get_alphas(df["$\\alpha$"].to_numpy(), df["$\\epsilon$"].to_numpy())
    df["$\\alpha_\\parallel$"] = alpha_par
    df["$\\alpha_\\perp$"] = alpha_perp

    # Get the MAP point and set the model up at this point
    model.set_data(data)
    r_s = model.camb.get_data()["r_s"]
    max_post = posterior.argmax()
    params = df.loc[max_post]
    params_dict = model.get_param_dict(chain[max_post])
    for name, val in params_dict.items():
        model.set_default(name, val)

    # Get some useful properties of the fit, and plot the MAP model against the data
    #display = True if "GCcomb" in extra["name"] else False
    figname = f"./plots/run_fits/{extra['name'].replace(' ', '_')}_bestfit.png" if "GCcomb" in extra["name"] else None
    new_chi_squared, dof, bband, mods, smooths = model.simple_plot(params_dict, display=False, figname=figname, c=cs[dataname_index])

    # Add the chain to the appropriate ChainConsumer instance
    if 'realisation' in extra.keys():
        extra.pop('realisation')
    cfull[dataname_index].add_chain(df, weights=weight, name=cap, plot_contour=True, plot_point=False, show_as_1d_prior=False)
    if "GCcomb" in extra["name"]:
        csmall[dataname_index].add_chain(df, weights=weight, name=cap, plot_contour=True, plot_point=False, show_as_1d_prior=False, color=cs[dataname_index])
    
    # Compute some summary statistics and add them to a dictionary
    mean, cov = weighted_avg_and_cov(
        df[
            [
                "$\\alpha$",
                "$\\epsilon$",
                "$\\alpha_\\parallel$",
                "$\\alpha_\\perp$",
                "$\\Sigma_{nl,||}$",
                "$\\Sigma_{nl,\\perp}$",
            ]
        ],
        weight,
        axis=0,
    )

    corr = cov[2, 3] / np.sqrt(cov[2, 2] * cov[3, 3])
    output.append(
        f"{extra['name'].replace(' ', '_'):s}, {mean[0]:6.4f}, {mean[1]:6.4f}, {mean[2]:6.4f}, {mean[3]:6.4f}, {mean[4]:6.4f}, {mean[5]:6.4f}, {np.sqrt(cov[0, 0]):6.4f}, {np.sqrt(cov[1, 1]):6.4f}, {np.sqrt(cov[2, 2]):6.4f}, {np.sqrt(cov[3, 3]):6.4f}, {corr:7.3f}, {r_s:7.3f}, {new_chi_squared:7.3f}, {dof:4d}"
    )

Plot the contour plots and output the summary statistics

In [17]:
%%capture
truth = {"$\\alpha$": 1.0, "$\\epsilon$": 0, "$\\alpha_\\perp$": 1.0, "$\\alpha_\\parallel$": 1.0}
for chain_bin in range(len(cfull)):
    print(datanames[chain_bin])
    figname = f"./plots/run_fits/fullchainplot_{datanames[chain_bin]}.png"
    cfull[chain_bin].plotter.plot(
        truth=truth,
        parameters=["$\\alpha$","$\\epsilon$","$\\alpha_\\parallel$", "$\\alpha_\\perp$","$\\Sigma_{nl,||}$","$\\Sigma_{nl,\\perp}$","$\\Sigma_s$"],
        filename=figname,
        figsize=(6,6)
    )
    
    figname = f"./plots/run_fits/smallchainplot_{datanames[chain_bin]}.png"
    csmall[chain_bin].plotter.plot(
        truth=truth,
        parameters=["$\\alpha_\\parallel$", "$\\alpha_\\perp$"],
        chains="GCcomb",
        filename=figname,
        figsize=(6,6)
    )
    
# Save all the numbers to a file
with open("./plots/run_fits/output/Barry_fits.txt", "w") as f:
    f.write(
        "# Name, alpha, epsilon, alpha_par, alpha_perp, Sigma_nl_par, Sigma_nl_perp, sigma_alpha_par, sigma_alpha_perp, corr_alpha_par_perp, rd_of_template, bf_chi2, dof\n"
    )
    for l in output:
        f.write(l + "\n")